In [1]:
# import packages
import pandas as pd
import numpy as np
import json
from pprint import pprint
import csv

In [5]:
# set maximum number of rows and columns displayed
pd.set_option('display.max_columns', 12)
pd.set_option('display.max_rows', 10)

# Specifies file to load into dataframe;primary pandas data structure   
df = pd.read_csv('Complete_Municipalities.csv')
df.head()

,Geo_Code,Prov_Name,CD_Name,CSD_Name,CSD_type,GNR,Topic,Characteristic,Note,Total,Male,Female
0,5901003,British Columbia,East Kootenay,Elkford,District municipality,49.0,Citizenship,Total population in private households by citi...,1.0,2520,1325,1195
1,5901003,British Columbia,East Kootenay,Elkford,District municipality,49.0,Citizenship,Canadian citizens,NaN,2400,1280,1120
2,5901003,British Columbia,East Kootenay,Elkford,District municipality,49.0,Citizenship,Canadian citizens aged under 18,NaN,570,275,295
3,5901003,British Columbia,East Kootenay,Elkford,District municipality,49.0,Citizenship,Canadian citizens aged 18 and over,NaN,1825,1000,825
4,5901003,British Columbia,East Kootenay,Elkford,District municipality,49.0,Citizenship,Not Canadian citizens,2.0,125,45,75


In [44]:
search_term = '    Average family size'
n_offset = 5   # number of rows prior to use

def find_offset_value(df, search_term, n_offset):
    ''' Finds the characteristic which is n rows above or below the desired search term.
    '''
    n = len(df['Characteristic'])
    
    # Initialize empty lists
    val = []   # median income value
    csd = []   # Geo_Code
    ctc = []   # characteristic 
    siz = []   # average family size

    for i in range(0,n):
        if(df['Characteristic'][i] == search_term):
            # Store data into appropriate lists if search term matches 
            csd.append(df['Geo_Code'][i-n_offset])         # take the row n_offset rows above
            ctc.append(df['Characteristic'][i-n_offset])
            
            # Try to convert value to integer
            try:
                val.append(int(df['Total'][i]))                # convert from string to integer; not sure which index should be used for this total value
            # If Null, can either fill with $0 or null
            except ValueError:
#                 val.append(0)   # fill with 0
                val.append(df['Total'][i])   # fill with null

    # Put the lists into a dictionary, which then will be returned in a dataframe
    data = {'Total': val,
            'Geo_Code': csd,
            'Characteristic': ctc}
    
    return pd.DataFrame(data)
    
df_sorted = find_offset_value(df, search_term, n_offset)
df_sorted.head()

,Characteristic,Geo_Code,Total
0,Couple-only economic families,5901003,2
1,Couple-with-children economic families,5901003,3.9
2,Lone-parent economic families,5901003,2.8
3,Couple-only economic families,5901006,2
4,Couple-with-children economic families,5901006,3.7


In [45]:
df_sorted.loc[df_sorted['Total']==0,'Total'] = 'Null'
df_sorted

,Characteristic,Geo_Code,Total
0,Couple-only economic families,5901003,2
1,Couple-with-children economic families,5901003,3.9
2,Lone-parent economic families,5901003,2.8
3,Couple-only economic families,5901006,2
4,Couple-with-children economic families,5901006,3.7
...,...,...,...
1306,Couple-with-children economic families,5959806,3.9
1307,Lone-parent economic families,5959806,3
1308,Couple-only economic families,5959810,Null
1309,Couple-with-children economic families,5959810,3.5


In [46]:
df_pivoted = pd.pivot_table(df_sorted, values='Total', index='Geo_Code', columns='Characteristic', aggfunc='first')
df_pivoted.to_csv('famSize_by_familyType.csv')

In [22]:
df_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1311 entries, 0 to 1310
Data columns (total 3 columns):
Characteristic    1311 non-null object
Geo_Code          1311 non-null int64
Total             1311 non-null object
dtypes: int64(1), object(2)
memory usage: 30.8+ KB
